In [3]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp
import yaml

import random
from itertools import chain, combinations, tee
import time


# Functions

In [38]:
def solve_od_single_freeway(num_nodes, origin_demands, destination_demands):
    
    assert num_nodes >= 3, "We must have num_nodes >= 3."
    assert origin_demands.shape[0] == num_nodes - 1, \
        "We must have origin_demands.shape[0] == num_nodes - 1."
    assert destination_demands.shape[0] >= num_nodes - 1, \
        "We must have destination_demands.shape[0] == num_nodes - 1."
    assert np.sum(origin_demands) == np.sum(destination_demands), \
        "We must have np.sum(origin_demands) == np.sum(destination_demands), to ensure flow continuity."
    
    ## TODO: Define T.
    
    T = {}
    func = 0.0
    
    # Variables:
    for i in range(num_nodes - 1):
        for j in range(i + 1, num_nodes):
            T[(i, j)] = cp.Variable(1)
            func += - cp.entr(T[(i, j)]) - T[(i, j)]
    
    objective = cp.Minimize(func)
    
    # Constraints:
    constraints = []
    constraints += [T[(i, j)] >= 0 for i in range(num_nodes - 1) for j in range(i + 1, num_nodes)]
    constraints += [sum(T[(i, j)] for j in range(i + 1, num_nodes)) == origin_demands[i] \
                    for i in range(num_nodes - 1)]
    constraints += [sum(T[(i, j)] for i in range(j)) == destination_demands[j-1] \
                    for j in range(1, num_nodes)]
    
    # Problem
    prob = cp.Problem(objective, constraints)
    
    # Solve:
    result = prob.solve()
    
    T_values = {}
    for i in range(num_nodes - 1):
        for j in range(i + 1, num_nodes):
            T_values[(i, j)] = max(T[(i, j)].value[0], 0.0)
    
    return T_values

def solve_od_general_network():

    return


In [39]:
# list(range(4, 5))

# Main

In [40]:
num_nodes = 3
origin_demands = np.array([2.0, 1.0])
destination_demands = np.array([1.2, 1.8])

solve_od_single_freeway(num_nodes, origin_demands, destination_demands)

{(0, 1): 1.2, (0, 2): 0.8, (1, 2): 1.0}

# Scratch Work

In [15]:
# Entropy function, test:

x = cp.Variable(10)
func = - cp.sum(cp.entr(x))
objective = cp.Minimize(func)
constraints = [x >= 0.0, cp.sum(x) == 1.0]
prob = cp.Problem(objective, constraints)

result = prob.solve()

x.value


array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])